In [1]:
import plotly as plotly
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from collections import Counter
import math
import datetime as datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import statistics
import glob
import ipyplot
import numpy as np
from functions import data_preprocess_functions
from matplotlib.colors import LogNorm

from matplotlib.image import NonUniformImage
import matplotlib.pyplot as plt


In [2]:
dir_name = 'idl_plots/output_sc1_sp3_300sec_multi_pa2_en2_subtraction_reduced_removebi_flux0.500.751.00_pap3.02.01.1/'
beam_filenames = sorted(glob.glob(dir_name+"data/*beam.csv"))
external_filenames = sorted(glob.glob(dir_name+"data/*external.csv"))

# take out 2016 data
beam_filenames = [x for x in beam_filenames if "2016" not in x]
external_filenames = [x for x in external_filenames if "2016" not in x]

In [3]:
# use those code if only compressed data are used
index_start = [i for i in range(len(beam_filenames)) if '20190416' in external_filenames[i]]
index_end = [i for i in range(len(beam_filenames)) if '20190816' in external_filenames[i]]

beam_filenames = beam_filenames[slice((index_start[0]+1),(index_end[0]+1))]

index_start = [i for i in range(len(external_filenames)) if '20190416' in external_filenames[i]]
index_end = [i for i in range(len(external_filenames)) if '20190816' in external_filenames[i]]

external_filenames = external_filenames[slice((index_start[0]+1),(index_end[0]+1))]

IndexError: list index out of range

In [4]:
df_ext = data_preprocess_functions.read_external_csv(external_filenames)

df_ext.rename(columns={'TIME':'time','BX_GSM':'bx','DIST':'dist', 'GSE_X':'xgse', 'GSE_Y':'ygse','GSE_Z':'zgse','GSM_X':'xgsm', 'GSM_Y':'ygsm','GSM_Z':'zgsm', 'BETA':'beta', 'O_V':'velocity_o_all', 'O_VPAR':'v_par_all', 'O_VPERP':'v_perp_all', 'O_N':'density_o_all', 'O_P':'pressure_o_all', 'H_N':'density_h_all', 'H_V':'velocity_h_all', 'KP':'kp', 'DST':'dst','MLT':'mlt'}, inplace = True)
    
df_ext_beam = df_ext.loc[(df_ext['FLAG_PARA'] == 1 ) | (df_ext['FLAG_ANTI'] == 1 ),:]

ValueError: No objects to concatenate

In [5]:
df = data_preprocess_functions.read_from_raw_csv(beam_filenames, external_filenames)
agg_df0 = data_preprocess_functions.aggregate_angle(df)
agg_df = data_preprocess_functions.aggregate_energy(agg_df0)

# agg_df.loc[:,'normalized_flux'] = (agg_df.loc[:,'intergrated_flux']/agg_df.loc[:,'B'])* statistics.mean(agg_df['B'])

ValueError: No objects to concatenate

In [6]:
index = (agg_df['region'] == 'Lobe' ) & (agg_df['flux'] > 0) & (abs(agg_df['ygsm']) < 15)

agg_df.loc[index,'datetime_str']

NameError: name 'agg_df' is not defined

In [7]:
# ex_beam_filename = [x for x in beam_filenames if "20170519" in x]
# ex_external_filename = [x for x in external_filenames if "20170519" in x]

# ex_df_beam = data_preprocess_functions.read_multiple_csv([ex_beam_filename[0]])
# ex_df_external = data_preprocess_functions.read_multiple_csv([ex_external_filename[0]])
# ex_df_beam.columns

# index = (ex_df_beam['Time'] == 1495170750) & (ex_df_beam['En'] == 1338)

# ex_df_beam.loc[index, ['Time','Pa_para','Flux_para','En']]

index = (agg_df['region'] == 'Lobe' ) & (agg_df['flux'] > 0) & (agg_df['date'] == '2019-06-01')
         
agg_df.loc[index, ['datetime_str','flux','int_flux','pa_range','energy','denergy','density_est','beta','ygsm','density_o_all']]

NameError: name 'agg_df' is not defined

In [8]:
index = (agg_df.density_est > 0) & (agg_df['region'] != 'Dayside') & (abs(agg_df['ygsm']) < 15)

fig = px.histogram(agg_df.loc[index,:], x=np.log10(agg_df.loc[index,"density_est"]) ,  facet_col = 'region'                )

fig.for_each_xaxis(lambda xaxis: xaxis.update(dict(
    tickvals=[-5,-4,-3,-2],
    ticktext=["0.00001", "0.0001", "0.001", "0.01"]
)))

fig.show()


NameError: name 'agg_df' is not defined

In [9]:
index = (agg_df.density_est > 0) & (agg_df['region'] != 'Dayside')

fig = px.scatter(agg_df.loc[index,:], x='xgsm', y='zgsm', color=np.log10(agg_df.loc[index,"density_est"]),range_color=[-5,-1], facet_col = 'region'
                )
fig.update_layout(coloraxis_colorbar=dict(
    title="density",
    tickvals=[-5,-4,-3,-2],
    ticktext=["0.00001", "0.0001", "0.001", "0.01"],
))

fig.update_traces(marker=dict(size=2), selector=dict(mode='markers'))

fig.show()


NameError: name 'agg_df' is not defined

In [10]:
index = (agg_df.intergrated_flux > 0) & (agg_df['region'] != 'Dayside') & (abs(agg_df['ygsm']) < 20)

fig = px.scatter(agg_df.loc[index,:], x='xgsm', y='density_est', color='region', log_y=True
                , category_orders={"region":["Lobe","BL","PS"]}
#                 , trendline="ols", trendline_options=dict(log_y=True)
                )
fig.update_yaxes(range=[-5, -1])
fig.update_traces(marker=dict(size=2), selector=dict(mode='markers'))

fig.show()

NameError: name 'agg_df' is not defined

In [ ]:
xedges = range(-30,10,2)
yedges = range(-20,20,2)

x_label = 'xgsm'
y_label = 'ygsm'

samples = data_preprocess_functions.calculate_occurrence_rate(df_ext[x_label],df_ext[y_label],xedges,yedges)

events = data_preprocess_functions.calculate_occurrence_rate(df_ext_beam[x_label],df_ext_beam[y_label],xedges,yedges) 

occurrence_rate=np.divide(events,samples)

intergrated_flux = data_preprocess_functions.calculate_density_heatmap(agg_df[x_label],agg_df[y_label],agg_df['intergrated_flux'],xedges,yedges)       

avg_flux = np.multiply(intergrated_flux,occurrence_rate)

density = data_preprocess_functions.calculate_density_heatmap(agg_df[x_label],agg_df[y_label],agg_df['density_est'],xedges,yedges)

avg_density = np.multiply(density,occurrence_rate)


In [ ]:
plt.figure(figsize=(14,7))

plt.subplot(231)
pos = plt.imshow(occurrence_rate, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='jet')

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('Occurrence frequency', rotation=90)

plt.subplot(232)
pos = plt.imshow(intergrated_flux, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('Intergrated flux', rotation=90)

plt.subplot(233)
pos = plt.imshow(density, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e-4, vmax=1e-2))

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('Density', rotation=90)

plt.subplot(235)
pos = plt.imshow(avg_flux, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('avg flux', rotation=90)

plt.subplot(236)
pos = plt.imshow(avg_density, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e-4, vmax=1e-2))

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('avg density', rotation=90)

plt.show()

In [ ]:
index_lobe = df_ext['beta'] <= 0.05

samples_lobe = data_preprocess_functions.calculate_occurrence_rate(df_ext.loc[index_lobe,x_label],df_ext.loc[index_lobe,y_label],xedges,yedges)

events_lobe = data_preprocess_functions.calculate_occurrence_rate(df_ext_beam.loc[index_lobe,x_label],df_ext_beam.loc[index_lobe,y_label],xedges,yedges)

occurrence_rate_lobe = np.divide(events_lobe,samples_lobe)

index_bl = (df_ext['beta'] > 0.05) & (df_ext['beta'] <= 1)

samples_bl = data_preprocess_functions.calculate_occurrence_rate(df_ext.loc[index_bl,x_label],df_ext.loc[index_bl,y_label],xedges,yedges)

events_bl = data_preprocess_functions.calculate_occurrence_rate(df_ext_beam.loc[index_bl,x_label],df_ext_beam.loc[index_bl,y_label],xedges,yedges)

occurrence_rate_bl = np.divide(events_bl,samples_bl)

index_ps = df_ext['beta'] > 1

samples_ps = data_preprocess_functions.calculate_occurrence_rate(df_ext.loc[index_ps,x_label],df_ext.loc[index_ps,y_label],xedges,yedges)

events_ps = data_preprocess_functions.calculate_occurrence_rate(df_ext_beam.loc[index_ps,x_label],df_ext_beam.loc[index_ps,y_label],xedges,yedges)

occurrence_rate_ps = np.divide(events_ps, samples_ps)


In [ ]:
index = agg_df['region'] == 'Lobe'

intergrated_flux_lobe = data_preprocess_functions.calculate_density_heatmap(agg_df.loc[index,x_label],agg_df.loc[index,y_label],agg_df.loc[index,'intergrated_flux'],xedges,yedges)       

avg_flux_lobe = np.multiply(intergrated_flux_lobe,occurrence_rate_lobe)

index = agg_df['region'] == 'BL'

intergrated_flux_bl = np.multiply(data_preprocess_functions.calculate_density_heatmap(agg_df.loc[index,x_label],agg_df.loc[index,y_label],agg_df.loc[index,'intergrated_flux'],xedges,yedges), occurrence_rate_lobe)

avg_flux_bl = np.multiply(intergrated_flux_bl , occurrence_rate_bl)

index = agg_df['region'] == 'PS'

intergrated_flux_ps = np.multiply(data_preprocess_functions.calculate_density_heatmap(agg_df.loc[index,x_label],agg_df.loc[index,y_label],agg_df.loc[index,'intergrated_flux'],xedges,yedges), occurrence_rate_lobe)

avg_flux_ps = np.multiply(intergrated_flux_ps , occurrence_rate_ps)

In [ ]:
plt.figure(figsize=(13,10))

plt.subplot(331)
pos = plt.imshow(occurrence_rate_lobe, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',vmin=0,vmax=1)
plt.title('Lobe', y=1.02, fontsize=12)

plt.subplot(332)
pos = plt.imshow(occurrence_rate_bl, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',vmin=0,vmax=1)
plt.title('BL', y=1.02, fontsize=12)

plt.subplot(333)
pos = plt.imshow(occurrence_rate_ps, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',vmin=0,vmax=1)
plt.title('PS', y=1.02, fontsize=12)

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('occurrence rate', rotation=90)

plt.subplot(334)
pos = plt.imshow(intergrated_flux_lobe, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))
plt.title('Lobe', y=1.02, fontsize=12)

plt.subplot(335)
pos = plt.imshow(intergrated_flux_bl, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))
plt.title('BL', y=1.02, fontsize=12)

plt.subplot(336)
pos = plt.imshow(intergrated_flux_ps, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))
plt.title('PS', y=1.02, fontsize=12)

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('intergrated flux', rotation=90)


plt.subplot(337)
pos = plt.imshow(avg_flux_lobe, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))
plt.title('Lobe', y=1.02, fontsize=12)

plt.subplot(338)
pos = plt.imshow(avg_flux_bl, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))
plt.title('BL', y=1.02, fontsize=12)

plt.subplot(339)
pos = plt.imshow(avg_flux_ps, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=1e3, vmax=1e5))
plt.title('PS', y=1.02, fontsize=12)

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('avg flux', rotation=90)


plt.show()


In [ ]:
index = agg_df['region'] == 'Lobe'

density_lobe = data_preprocess_functions.calculate_density_heatmap(agg_df.loc[index,x_label],agg_df.loc[index,y_label],agg_df.loc[index,'density_est'],xedges,yedges)

avg_density_lobe = np.multiply(density_lobe , occurrence_rate_lobe)

index = agg_df['region'] == 'BL'

density_bl = np.multiply(data_preprocess_functions.calculate_density_heatmap(agg_df.loc[index,x_label],agg_df.loc[index,y_label],agg_df.loc[index,'density_est'],xedges,yedges), occurrence_rate_lobe)

avg_density_bl = np.multiply(density_bl , occurrence_rate_bl)

index = agg_df['region'] == 'PS'

density_ps = np.multiply(data_preprocess_functions.calculate_density_heatmap(agg_df.loc[index,x_label],agg_df.loc[index,y_label],agg_df.loc[index,'density_est'],xedges,yedges), occurrence_rate_lobe)

avg_density_ps = np.multiply(density_ps , occurrence_rate_ps)


In [ ]:
plt.figure(figsize=(13,10))

plt.subplot(331)
pos = plt.imshow(occurrence_rate_lobe, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',vmin=0,vmax=1)
plt.title('Lobe', y=1.02, fontsize=12)

plt.subplot(332)
pos = plt.imshow(occurrence_rate_bl, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',vmin=0,vmax=1)
plt.title('BL', y=1.02, fontsize=12)

plt.subplot(333)
pos = plt.imshow(occurrence_rate_ps, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',vmin=0,vmax=1)
plt.title('PS', y=1.02, fontsize=12)

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('occurrence rate', rotation=90)

plt.subplot(334)
pos = plt.imshow(density_lobe, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=0.0001, vmax=0.01))
plt.title('Lobe', y=1.02, fontsize=12)

plt.subplot(335)
pos = plt.imshow(density_bl, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=0.0001, vmax=0.01))
plt.title('BL', y=1.02, fontsize=12)

plt.subplot(336)
pos = plt.imshow(density_ps, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=0.0001, vmax=0.01))
plt.title('PS', y=1.02, fontsize=12)

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('density', rotation=90)


plt.subplot(337)
pos = plt.imshow(avg_density_lobe, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=0.0001, vmax=0.01))
plt.title('Lobe', y=1.02, fontsize=12)

plt.subplot(338)
pos = plt.imshow(avg_density_bl, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=0.0001, vmax=0.01))
plt.title('BL', y=1.02, fontsize=12)

plt.subplot(339)
pos = plt.imshow(avg_density_ps, interpolation='none', origin='lower',extent=[xedges[0], xedges[-1]+xedges.step, yedges[0], yedges[-1]+yedges.step], cmap='jet',norm=LogNorm(vmin=0.0001, vmax=0.01))
plt.title('PS', y=1.02, fontsize=12)

cbar = plt.colorbar(pos)
cbar.ax.set_ylabel('avg density', rotation=90)


plt.show()


In [ ]:
data = pd.DataFrame({'region':['Lobe','BL','PS'], 'near':[0,0,0],'tail':[0,0,0]})

for i in range(len(data.region)):
    index1 = (agg_df.flux > 0) & (agg_df['xgsm'] > -15) & (agg_df['region'] == data.region[i])
    index2 = (agg_df.flux > 0) & (agg_df['xgsm'] < -15) & (agg_df['region'] == data.region[i])
    data.loc[i,'near'] = round(statistics.mean(agg_df.loc[index1,'density_est']),4)
    data.loc[i,'tail'] = round(statistics.mean(agg_df.loc[index2,'density_est']),4)

index_near = (agg_df['beta'] > 0.04) & (agg_df['beta'] < 0.05) & (agg_df.density_est > 0) & (agg_df['xgsm'] > -15)
index_tail = (agg_df['beta'] > 0.04) & (agg_df['beta'] < 0.05) & (agg_df.density_est > 0) & (agg_df['xgsm'] < -15)
near_BL_Lobe_data = {'region':'near BL Lobe', 'near': round(statistics.mean(agg_df.loc[index_near,'density_est']),4), 'tail': round(statistics.mean(agg_df.loc[index_tail,'density_est']),4)}

data = data.append(near_BL_Lobe_data, ignore_index=True)

data 

In [25]:
0.0009e-6 # m-3
10e3 #m/s

"{:e}".format(0.0009e-6 * 10e3 * 10*6371e3 * 30*6371e3 * 2)

'2.191841e+11'

In [24]:
0.1 # cm-3
25 # km/s
30 #Re
4 #Re

"{:e}".format(0.04e-6 * 25e3 * 30*6371e3 * 4*6371e3)

'4.870757e+12'